In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import sys

PATH = '/content/drive/MyDrive/Colab Notebooks/CS598_DLH_Paper211'
sys.path.append(PATH)


Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

SEED = 310

In [ ]:
lvl2_train =  pd.read_pickle(PATH + "/data/lvl2_imputer_train.pkl")
lvl2_dev =  pd.read_pickle(PATH + "/data/lvl2_imputer_dev.pkl")
lvl2_test =  pd.read_pickle(PATH + "/data/lvl2_imputer_test.pkl")

Ys =  pd.read_pickle(PATH + "/data/Ys"+".pkl")
Ys_train =  pd.read_pickle(PATH + "/data/Ys_train.pkl")
Ys_dev =  pd.read_pickle(PATH + "/data/Ys_dev.pkl")
Ys_test =  pd.read_pickle(PATH + "/data/Ys_test.pkl")

In [ ]:
all_train_ids = set()
for i in Ys_train.itertuples():
    all_train_ids.add( i.Index[0] )
    
all_dev_ids = set()
for i in Ys_dev.itertuples():
    all_dev_ids.add( i.Index[0] )
    
all_test_ids = set()
for i in Ys_test.itertuples():
    all_test_ids.add( i.Index[0] )

In [ ]:
print (sum(Ys_train.mort_icu.values)*1.0 / len(Ys_train.mort_icu.values))
print (sum(Ys_dev.mort_icu.values)*1.0 / len(Ys_dev.mort_icu.values))
print (sum(Ys_test.mort_icu.values)*1.0 / len(Ys_test.mort_icu.values))
print ("====")
print (sum(Ys_train.mort_hosp.values)*1.0 / len(Ys_train.mort_hosp.values))
print (sum(Ys_dev.mort_hosp.values)*1.0 / len(Ys_dev.mort_hosp.values))
print (sum(Ys_test.mort_hosp.values)*1.0 / len(Ys_test.mort_hosp.values))
print ("====")
print (sum(Ys_train.los_3.values)*1.0 / len(Ys_train.los_3.values))
print (sum(Ys_dev.los_3.values)*1.0 / len(Ys_dev.los_3.values))
print (sum(Ys_test.los_3.values)*1.0 / len(Ys_test.los_3.values))
print ("====")
print (sum(Ys_train.los_7.values)*1.0 / len(Ys_train.los_7.values))
print (sum(Ys_dev.los_7.values)*1.0 / len(Ys_dev.los_7.values))
print (sum(Ys_test.los_7.values)*1.0 / len(Ys_test.los_7.values))

0.07159904534606205
0.06725146198830409
0.07432150313152401
====
0.10680190930787589
0.09649122807017543
0.10855949895615867
====
0.27852028639618137
0.2723475355054302
0.2659707724425887
====
0.07696897374701671
0.07268170426065163
0.07954070981210856


In [ ]:
def mean(a):
    return sum(a) / len(a)

In [ ]:
new_word2vec_dict = pd.read_pickle(PATH + "/data/new_ner_word2vec_limited_dict.pkl")
new_keys = set(new_word2vec_dict.keys())
new_train_ids = sorted(all_train_ids.intersection(new_keys))
new_dev_ids = sorted(all_dev_ids.intersection(new_keys))
new_test_ids = sorted(all_test_ids.intersection(new_keys))

In [ ]:

pd.to_pickle (new_train_ids, PATH + "/data/new_train_ids.pkl")
pd.to_pickle(new_dev_ids, PATH + "/data/new_dev_ids.pkl")
pd.to_pickle(new_test_ids, PATH + "/data/new_test_ids.pkl")

new_train_ids = pd.read_pickle(PATH + "/data/new_train_ids.pkl")
new_dev_ids = pd.read_pickle(PATH + "/data/new_dev_ids.pkl")
new_test_ids = pd.read_pickle(PATH + "/data/new_test_ids.pkl")


In [ ]:
data_ids = [(new_train_ids, new_dev_ids, new_test_ids)]
data_names = ["new"]

for i, (tr, de, te) in zip(data_names, data_ids):
    
    y_train = Ys_train.loc[tr]
    y_dev = Ys_dev.loc[de]
    y_test = Ys_test.loc[te]

    print(y_train.shape)

    sub_train = lvl2_train.loc[tr]
    sub_train = sub_train.loc[:, pd.IndexSlice[:, 'mean']]

    sub_dev = lvl2_dev.loc[de]
    sub_dev = sub_dev.loc[:, pd.IndexSlice[:, 'mean']]

    sub_test = lvl2_test.loc[te]
    sub_test = sub_test.loc[:, pd.IndexSlice[:, 'mean']]

    sub_train = sub_train.values
    sub_dev = sub_dev.values
    sub_test = sub_test.values

    print(sub_test.shape)

    re_shape_train = (int(sub_train.shape[0] / 24), 24, 104)
    re_shape_dev = (int(sub_dev.shape[0] / 24), 24, 104)
    re_shape_test = (int(sub_test.shape[0] / 24), 24, 104)
    # reshape the data for timeseries prediction
    x_train_lstm = sub_train.reshape(re_shape_train)
    x_dev_lstm = sub_dev.reshape(re_shape_dev)
    x_test_lstm = sub_test.reshape(re_shape_test)

    
    pd.to_pickle(x_train_lstm, PATH + "/data/"+i+"_x_train.pkl")
    pd.to_pickle(x_dev_lstm, PATH + "/data/"+i+"_x_dev.pkl")
    pd.to_pickle(x_test_lstm, PATH + "/data/"+i+"_x_test.pkl")
    
    pd.to_pickle(y_train, PATH + "/data/"+i+"_y_train.pkl")
    pd.to_pickle(y_dev, PATH + "/data/"+i+"_y_dev.pkl")
    pd.to_pickle(y_test, PATH + "/data/"+i+"_y_test.pkl")

(15219, 4)
(104352, 104)
